In [1]:
import gym
import pandas as pd
import numpy as np

# Environment

In [2]:
k = 10

flake = gym.make('FrozenLake-v1', is_slippery=False)
new_s = flake.reset()
flake.render()

/Users/hoya/hWorkspace/AI_Study/.venv/lib/python3.9/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/Users/hoya/hWorkspace/AI_Study/.venv/lib/python3.9/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/Users/hoya/hWorkspace/AI_Study/.venv/lib/python3.9/site-packages/gym/core.py:57: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize

In [3]:
def run(n_iter=100, render_flag=False):
    new_s = flake.reset()
    if render_flag:
        flake.render()
    buff_df = pd.DataFrame({"S": [new_s], "S: (x, y)": [(0, 0)], "R": [0.0], "done": [False], "A": [0], "A:name": [""]})
    buff_df.index_name = 'k'

    actions = ['Lest', 'Down', 'Right', 'Up']
    for iter in range(n_iter):
        a_k = flake.action_space.sample()
        buff_df.loc[iter, 'A': "A:name"] = (a_k, actions[a_k])
        s, r, done, info = flake.step(a_k)
        if render_flag:
            flake.render()
        new_df = pd.DataFrame({'S': [s], "S: (x, y)": [(s % 4, s // 4)], "R": [r], "done": [done], "A": [0], "A:name": [""]})
        buff_df = buff_df.append(new_df, ignore_index=True)
        buff_df.index.name = 'k'
        if done:
            return buff_df
    return None

In [4]:
def calc_g(r, factor=0.9):
    g_prev = 0
    g = np.copy(r[1:])
    g = np.append(g, 0.0)
    for rev_k in range(len(g) - 2, -1, -1):
        g[rev_k] += factor * g_prev
        g_prev = g[rev_k]
    return g

def get_g(n_iter=50):
    buff_df = run(n_iter)
    if buff_df is not None:
        r = buff_df.R.values
        buff_df['G'] = calc_g(r)
    else:
        print('Try more iterations for each run')
        return None
    return buff_df

def get_g_many(n_epochs=5, n_iter=50):
    gbuff_df = None
    for epoch in range(n_epochs):
        buff_df = get_g(n_iter)
        if buff_df is not None:
            if epoch == 0:
                gbuff_df = buff_df
            else:
                gbuff_df = gbuff_df.append(buff_df)
    return gbuff_df

# Draw Image

In [5]:
gbuff_df = get_g_many()
v = np.zeros(flake.observation_space.n)
n_v = np.zeros(flake.observation_space.n)
for s in range(flake.observation_space.n):
    gs_all = gbuff_df.G[gbuff_df.S == s].values
    if len(gs_all) > 0:
        v[s] = np.average(gs_all)
        n_v[s] = len(gs_all)
v_df = pd.DataFrame({"V": v, "No of Gs": n_v})
v_df.index.name = 's'

gbuff_df = get_g_many(100)
q = np.zeros((flake.observation_space.n, flake.action_space.n))
n_q = np.zeros((flake.observation_space.n, flake.action_space.n))
s_list = []
a_list = []
for s in range(flake.observation_space.n):
    for a in range(flake.action_space.n):
        gs_all = gbuff_df.G[(gbuff_df.S == s) & (gbuff_df.A == a)].values
        if len(gs_all) > 0:
            q[s, a] = np.average(gs_all)
            n_q[s, a] = len(gs_all)
        s_list.append(s)
        a_list.append(a)
sa_df = pd.DataFrame({"S": s_list, "A": a_list})
q_df = pd.DataFrame({"Q": q.reshape(-1), "No of Gs": n_q.reshape(-1)}, index=pd.MultiIndex.from_frame(sa_df))

/var/folders/b5/gg8xwlms3jj_xtngfs9m63pw0000gn/T/ipykernel_11603/541296619.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  buff_df = buff_df.append(new_df, ignore_index=True)
/var/folders/b5/gg8xwlms3jj_xtngfs9m63pw0000gn/T/ipykernel_11603/541296619.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  buff_df = buff_df.append(new_df, ignore_index=True)
/var/folders/b5/gg8xwlms3jj_xtngfs9m63pw0000gn/T/ipykernel_11603/3351620872.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gbuff_df = gbuff_df.append(buff_df)
/var/folders/b5/gg8xwlms3jj_xtngfs9m63pw0000gn/T/ipykernel_11603/541296619.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

In [6]:
pi = np.argmax(q, axis=1)
pi.reshape(4, 4)

def run_with_pi(pi=None, n_iter=100, render_flag=False):
    s = flake.reset()
    if render_flag:
        flake.render()
    buff_df = pd.DataFrame({"S": [s], "S:(x, y)": [(0, 0)], "R": [0.0], "done": [False], "A": [0], "A:name": [""]})
    buff_df.index.name = 'k'
    actions = ['Left', 'Down', 'Right', 'Up']
    for iter in range(n_iter):
        if pi is not None:
            a_k = pi[s]
        else:
            a_k = flake.action_space.sample()
        buff_df.loc[iter, "A":"A:name"] = (a_k, actions[a_k])
        s, r, done, info = flake.step(a_k)
        if render_flag:
            flake.render()
        new_df = pd.DataFrame({"S": [s], "S:(x, y)": [(s % 4, s // 4)], "R": [r], "done": [done], "A": [0], "A:name": [""]})
        buff_df = buff_df.append(new_df, ignore_index=True)
        buff_df.index.name = 'k'
        if done:
            return buff_df
    return None

run_with_pi(pi=pi, n_iter=1, render_flag=True)

/Users/hoya/hWorkspace/AI_Study/.venv/lib/python3.9/site-packages/gym/core.py:57: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/var/folders/b5/gg8xwlms3jj_xtngfs9m63pw0000gn/T/ipykernel_11603/2933958382.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  buff_df = buff_df.append(new_df, ignore_index=True)


: 